1. список детекторов
1. поиск тх для детекторов
1. скоры для всех детекторов

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import lightgbm as lgb
import seaborn as sns

In [2]:
from river import anomaly
from river import stream
from river import evaluate
from river import metrics
from river import compose
from river import preprocessing

In [3]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

In [4]:
PROJECT_ROOT                     = "../../"

SOURCE_DATASET_ROOT              = PROJECT_ROOT + "datasets/isp3/"

SOURCE_DATASET_FILENAME          = SOURCE_DATASET_ROOT + "[dvwa.isp_xvwa.isp]_https_apache.csv"
SOURCE_DATASET_FILENAME_D        = SOURCE_DATASET_ROOT + "[dvwa.isp]_https_apache.csv"
SOURCE_DATASET_FILENAME_X        = SOURCE_DATASET_ROOT + "[xvwa.isp]_https_apache.csv"

SOURCE_DATASET_FILENAME_ADD      = SOURCE_DATASET_ROOT + "[xvwa.isp]_https_apache_xss_xsstrike_[5][1-0][delay0ms][-]_[2024-05-22][13.36.48].csv"
# SOURCE_DATASET_TRAIN_FILENAME    = SOURCE_DATASET_ROOT + "DataSetN_Train.csv"
# SOURCE_DATASET_TEST_FILENAME     = SOURCE_DATASET_ROOT + "DataSetN_Test.csv"

# Фичи

In [5]:
df = pd.read_csv(SOURCE_DATASET_FILENAME)
df['Label'].value_counts()

/tmp/ipykernel_9173/3322869287.py:1: DtypeWarning: Columns (3,103,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(SOURCE_DATASET_FILENAME)


Label
Web Attack - DDoS                 65295
Benign                             4446
Web Attack - XSS                   2686
Web Attack - SQL Injection         1483
Web Attack - Command Injection     1400
Web Attack - Brute Force            497
Web Attack - Web Shell              353
Web Attack - CSRF                    60
Name: count, dtype: int64

In [6]:
excluded = ['Flow_ID', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port', 'Protocol', 'Timestamp', "Unnamed: 0", "xvwa.isp", "Label", "dvwa.isp"]
df = df.drop(columns=excluded, errors='ignore')

In [7]:
excluded2 = ['Session_Index',  'Target','Http_Reqest','Root','GlobalLabel','Type','Tools','ToolsThreads','ToolsDelay','ToolsAdditional',
             'ChannelSpeedBefore','ChannelSpeedAfter','NetemString','Server','KeepAliveTimeout','TargetProtocol','File','SessionAnalizerMode']
df = df.drop(columns=excluded2)

In [8]:
webattack_features = list(df.columns)

In [9]:
df = pd.read_csv(SOURCE_DATASET_FILENAME)
df['Label'].value_counts()

/tmp/ipykernel_9173/3322869287.py:1: DtypeWarning: Columns (3,103,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(SOURCE_DATASET_FILENAME)


Label
Web Attack - DDoS                 65295
Benign                             4446
Web Attack - XSS                   2686
Web Attack - SQL Injection         1483
Web Attack - Command Injection     1400
Web Attack - Brute Force            497
Web Attack - Web Shell              353
Web Attack - CSRF                    60
Name: count, dtype: int64

In [10]:
df_add = pd.read_csv(SOURCE_DATASET_FILENAME_ADD)
df_add["Label"] = df_add["Label"] + " - xsstrike"
df = pd.concat([df, df_add])
df["Label"].value_counts()

Label
Web Attack - DDoS                 65295
Benign                             4446
Web Attack - XSS - xsstrike        3077
Web Attack - XSS                   2686
Web Attack - SQL Injection         1483
Web Attack - Command Injection     1400
Web Attack - Brute Force            497
Web Attack - Web Shell              353
Web Attack - CSRF                    60
Name: count, dtype: int64

In [11]:
df.loc[df["Label"] == "Web Attack - XSS", "Label"] += " - xsser"
df["Label"].value_counts()

Label
Web Attack - DDoS                 65295
Benign                             4446
Web Attack - XSS - xsstrike        3077
Web Attack - XSS - xsser           2686
Web Attack - SQL Injection         1483
Web Attack - Command Injection     1400
Web Attack - Brute Force            497
Web Attack - Web Shell              353
Web Attack - CSRF                    60
Name: count, dtype: int64

In [12]:
label_rows = df[df['Label'] == 'Web Attack - DDoS']
rows_to_remove = label_rows.sample(frac=0.94, random_state=42)
df = df.drop(index=rows_to_remove.index)
df['Label'].value_counts()

Label
Benign                            4446
Web Attack - DDoS                 3918
Web Attack - XSS - xsser          2686
Web Attack - XSS - xsstrike       2319
Web Attack - SQL Injection        1483
Web Attack - Command Injection    1400
Web Attack - Brute Force           497
Web Attack - Web Shell             353
Web Attack - CSRF                   60
Name: count, dtype: int64

In [13]:
df['Label'].value_counts()

Label
Benign                            4446
Web Attack - DDoS                 3918
Web Attack - XSS - xsser          2686
Web Attack - XSS - xsstrike       2319
Web Attack - SQL Injection        1483
Web Attack - Command Injection    1400
Web Attack - Brute Force           497
Web Attack - Web Shell             353
Web Attack - CSRF                   60
Name: count, dtype: int64

In [14]:
df = df[webattack_features + ["Label"]].dropna()

In [15]:
attacks = {
    'Web Attack - XSS - xsser',
    "Web Attack - XSS - xsstrike",
    'Web Attack - CSRF',
    'Web Attack - Brute Force',
    'Web Attack - Web Shell',
    'Web Attack - Command Injection',
    'Web Attack - SQL Injection',
    'Web Attack - DDoS'
}

attack_to_exclude = {'Web Attack - XSS - xsstrike'}

benign = {'Benign', 'FromInSide'}

In [16]:
df['Label'] = df['Label'].replace(to_replace=benign, value='Benign')
df_full = df[df['Label'].isin(attacks | benign)]
df_part = df_full[~df_full['Label'].isin(attack_to_exclude)]

In [17]:
def prepare_dataset(df, train_size=0.2, test_size=0.8):
    X = df[webattack_features]
    y = df['Label']
    if train_size == 0 and test_size == 1:
        return [], X, [], y
    else:
        X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, 
                                                            train_size=train_size, test_size=test_size, shuffle=True, random_state=0)
    return X_train, X_test, y_train, y_test

In [18]:
_, X_test1, _, y_test1 = prepare_dataset(df_part, train_size=0, test_size=1)

In [19]:
_, X_test2, _, y_test2 = prepare_dataset(df_full, train_size=0, test_size=1)

In [20]:
df_part["Label"].value_counts()

Label
Benign                            4446
Web Attack - DDoS                 3918
Web Attack - XSS - xsser          2686
Web Attack - SQL Injection        1483
Web Attack - Command Injection    1400
Web Attack - Brute Force           497
Web Attack - Web Shell             353
Web Attack - CSRF                   60
Name: count, dtype: int64

In [21]:
df_full["Label"].value_counts()

Label
Benign                            4446
Web Attack - DDoS                 3918
Web Attack - XSS - xsser          2686
Web Attack - XSS - xsstrike       2319
Web Attack - SQL Injection        1483
Web Attack - Command Injection    1400
Web Attack - Brute Force           497
Web Attack - Web Shell             353
Web Attack - CSRF                   60
Name: count, dtype: int64

# Границы

In [22]:
def add_result(result, key, val):
    if key in result:
        result[key].append(val)
    else:
        result[key] = [val]

In [23]:
def test(df, attacks, benign, test_ood):
    df_full = df[df['Label'].isin(attacks | benign)]
    result = {"Label": []}
    for attack_to_exclude in tqdm(["Web Attack - XSS - xsstrike", "Web Attack - XSS - xsser"]):
        report, name = test_ood(df_full, {attack_to_exclude})
        
        result["Label"].append(attack_to_exclude)
        # add_result(result, name, f"{report._f1s[True].recall.get()*100}/{report._f1s[False].get()*100}")
        add_result(result, "recall for anomaly", report._f1s[True].recall.get())
        add_result(result, "f1 for normal", report._f1s[False].get())
    result = pd.DataFrame(result)
    result = result.set_index("Label")
    columns=[(name, c) for c in result.columns]
    result.columns=pd.MultiIndex.from_tuples(columns)
    return result

In [24]:
def test_ood_OneClassSVM(df, attack_to_exclude):
    def comp_threshold(scores: np.ndarray):
        for i in np.linspace(0.0, 0.9, num=50):
            res = (scores<i).sum()/scores.shape[0]
            if res>0.95:
                return i
    
    def prepare_df(df):
        X = df[webattack_features]
        y = df['Label']
        MinMaxScaler(df[webattack_features])
        return X, y

    def fit(df):
        X, y = prepare_df(df)
        model = OneClassSVM(nu=0.0021468216988400914, kernel='linear', gamma='auto')
        model = model.fit(X.to_numpy())
    
        scores = model.score_samples(X.to_numpy())
        th = comp_threshold(scores)
        return model, th
    
    df_part = df[~df['Label'].isin(attack_to_exclude)]
    model, th = fit(df_part)
    
    X, y = prepare_df(df)
    
    report = metrics.ClassificationReport()
    result = {"Label": [], "Score": [], "is_anomaly": []}
    for x, yi1 in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        x = np.array(list(x.values())).reshape(1, -1)
        result["Label"].append(yi1)
        is_anomaly = model.predict(x)[0] == -1
        result["is_anomaly"].append(is_anomaly)
        report.update(yi1 in attack_to_exclude, is_anomaly)
        
    print(report)
    return report, "OCSVM"

In [25]:
def test_ood_LocalOutlierFactor(df, attack_to_exclude):
    def prepare_df(df):
        X = df[webattack_features]
        y = df['Label']
        return X, y
    
    def fit(df):
        X, y = prepare_df(df)
        model = LocalOutlierFactor(novelty=True)
        # for x, yil in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        #     x = np.array(list(x.values())).reshape(1, -1)
        #     model = model.fit(x)
        model = model.fit(X.to_numpy())
        return model
    
    df_part = df[~df['Label'].isin(attack_to_exclude)]
    model = fit(df_part)
    
    X, y = prepare_df(df)
    
    report = metrics.ClassificationReport()
    result = {"Label": [], "Score": [], "is_anomaly": []}
    for x, yi1 in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        x = np.array(list(x.values())).reshape(1, -1)
        result["Label"].append(yi1)
        is_anomaly = model.predict(x)[0] == -1
        result["is_anomaly"].append(is_anomaly)
        report.update(yi1 in attack_to_exclude, is_anomaly)
        
    print(report)
    return report, "LOF"

In [26]:
def test_ood_IsolationForest(df, attack_to_exclude):
    def prepare_df(df):
        X = df[webattack_features]
        y = df['Label']
        return X, y
    
    def fit(df):
        X, y = prepare_df(df)
        model = IsolationForest()
        # for x, yil in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        #     x = np.array(list(x.values())).reshape(1, -1)
        #     model = model.fit(x)
        model = model.fit(X.to_numpy())
        return model
    
    df_part = df[~df['Label'].isin(attack_to_exclude)]
    model = fit(df_part)
    
    X, y = prepare_df(df)
    
    report = metrics.ClassificationReport()
    result = {"Label": [], "Score": [], "is_anomaly": []}
    for x, yi1 in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        x = np.array(list(x.values())).reshape(1, -1)
        result["Label"].append(yi1)
        is_anomaly = model.predict(x)[0] == -1
        result["is_anomaly"].append(is_anomaly)
        report.update(yi1 in attack_to_exclude, is_anomaly)
        
    print(report)
    return report, "IF"

In [27]:
class AnsambleOOD:
    def __init__(self, models: list):
        self.models = models
        
    def fit(self, X):
        for model in self.models:
            model.fit(X)
        return self
            
    def predict(self, X):
        scores = np.zeros(X.shape[0])
        for model in self.models:
            scores += model.predict(X)
        return scores

In [28]:
def test_ood_AnsambleOOD(df, attack_to_exclude):
    def prepare_df(df):
        X = df[webattack_features]
        y = df['Label']
        return X, y
    
    def fit(df):
        X, y = prepare_df(df)
        model = AnsambleOOD([
            OneClassSVM(kernel='rbf', gamma='auto', nu=0.01),
            LocalOutlierFactor(novelty=True),
            IsolationForest()
        ])
        model = model.fit(X.to_numpy())
        return model
    
    df_part = df[~df['Label'].isin(attack_to_exclude)]
    model = fit(df_part)
    
    X, y = prepare_df(df)
    
    report = metrics.ClassificationReport()
    result = {"Label": [], "Score": [], "is_anomaly": []}
    for x, yi1 in tqdm(stream.iter_pandas(X, y), total=y.shape[0], leave=False):
        x = np.array(list(x.values())).reshape(1, -1)
        result["Label"].append(yi1)
        is_anomaly = model.predict(x)[0] < 0
        result["is_anomaly"].append(is_anomaly)
        report.update(yi1 in attack_to_exclude, is_anomaly)
        
    print(report)
    return report, "Ensemble"

In [29]:
result1 = test(df, attacks, benign, test_ood_OneClassSVM)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      86.47%   99.78%   92.65%     14843  
    True       5.88%    0.09%    0.17%      2319  
                                                  
   Macro      46.18%   49.94%   46.41%            
   Micro      86.31%   86.31%   86.31%            
Weighted      75.58%   86.31%   80.16%            

                 86.31% accuracy                  


  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      84.34%   99.77%   91.41%     14476  
    True      13.16%    0.19%    0.37%      2686  
                                                  
   Macro      48.75%   49.98%   45.89%            
   Micro      84.19%   84.19%   84.19%            
Weighted      73.20%   84.19%   77.16%            

                 84.19% accuracy                  


In [30]:
result2 = test(df, attacks, benign, test_ood_IsolationForest)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      85.58%   92.70%   88.99%     14843  
    True       0.00%    0.00%    0.00%      2319  
                                                  
   Macro      42.79%   46.35%   44.50%            
   Micro      80.17%   80.17%   80.17%            
Weighted      74.01%   80.17%   76.97%            

                 80.17% accuracy                  


  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      83.36%   92.42%   87.66%     14476  
    True       1.44%    0.60%    0.84%      2686  
                                                  
   Macro      42.40%   46.51%   44.25%            
   Micro      78.05%   78.05%   78.05%            
Weighted      70.54%   78.05%   74.07%            

                 78.05% accuracy                  


In [31]:
result3 = test(df, attacks, benign, test_ood_LocalOutlierFactor)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      87.30%   89.78%   88.52%     14843  
    True      20.03%   16.39%   18.03%      2319  
                                                  
   Macro      53.66%   53.08%   53.27%            
   Micro      79.86%   79.86%   79.86%            
Weighted      78.21%   79.86%   79.00%            

                 79.86% accuracy                  


  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      94.76%   89.73%   92.18%     14476  
    True      56.96%   73.27%   64.09%      2686  
                                                  
   Macro      75.86%   81.50%   78.13%            
   Micro      87.15%   87.15%   87.15%            
Weighted      88.85%   87.15%   87.78%            

                 87.15% accuracy                  


In [32]:
result_a = test(df, attacks, benign, test_ood_AnsambleOOD)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      86.60%   84.40%   85.49%     14843  
    True      14.10%   16.39%   15.16%      2319  
                                                  
   Macro      50.35%   50.39%   50.32%            
   Micro      75.21%   75.21%   75.21%            
Weighted      76.80%   75.21%   75.98%            

                 75.21% accuracy                  


  0%|          | 0/17162 [00:00<?, ?it/s]

           Precision   Recall   F1       Support  
                                                  
   False      94.75%   84.95%   89.58%     14476  
    True      47.92%   74.65%   58.37%      2686  
                                                  
   Macro      71.34%   79.80%   73.98%            
   Micro      83.34%   83.34%   83.34%            
Weighted      87.42%   83.34%   84.70%            

                 83.34% accuracy                  


In [33]:
result_a

Ensemble              
                            recall for anomaly f1 for normal
Label                                                       
Web Attack - XSS - xsstrike           0.163864      0.854862
Web Attack - XSS - xsser              0.746463      0.895826

In [34]:
results_list = [result1, result2, result3, result_a]

In [35]:
results = pd.DataFrame()
results["Label"] = (result1.index).map(lambda x: x[13:])
for result in results_list:
    name = result.columns[0][0]
    print(name)
    a = (result[name]['recall for anomaly']*100).round(1)
    b = (result[name]['f1 for normal']*100).round(1)
    results[name] = a.combine(b, lambda x, y: f"{x}/{y}").reset_index(drop=True)
results

OCSVM
IF
LOF
Ensemble


,Label,OCSVM,IF,LOF,Ensemble
0,XSS - xsstrike,0.1/92.7,0.0/89.0,16.4/88.5,16.4/85.5
1,XSS - xsser,0.2/91.4,0.6/87.7,73.3/92.2,74.6/89.6


In [36]:
# result = pd.merge(result1, result2, left_index=True, right_index=True)
# result = pd.merge(result, result3, left_index=True, right_index=True)
# result = pd.merge(result, result_a, left_index=True, right_index=True)
# display(result)
# display(pd.DataFrame({"max": result.max(), "min": result.min(), "mean": result.mean()}))

In [37]:
# result = result.round(3)

In [38]:
# result.index = result.index.map(lambda s: s[13:])

In [39]:
results.to_csv('ood-results-xsser.csv', index=False)

In [40]:
results = pd.read_csv('ood-results-xsser.csv', header=[0, 1], index_col=0)